# Homework 2 -Data from the Web

Useful imports first

In [23]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import html5lib 
from lxml import html
import numpy as np

We want to get first all the parameters that are possible, i.e. we will fill two dictionnaries, one with the year as index, and the "value" of the link associated to it (e.g. {'2016-2017': '355925344'}), and the same with the bachelor year and the "value" of the link associated to it.

We do that in order to be able to extract all students from all possible years (2007-2016) and from all stages of the bachelor automatically later.

In [85]:
# The URL of the "start" of the IS-Academia page listing the students.
home_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_x_GPS=-1&ww_i_reportModel=133685247"

# The general form of the URL, with the fields to be replaced later (e.g. [UNITE_ACADEMIQUE])
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=[UNITE_ACADEMIQUE]&ww_x_PERIODE_ACAD=[PERIODE_ACADEMIQUE]&ww_x_PERIODE_PEDAGO=[PERIODE_PEDAGOGIQUE]&ww_x_HIVERETE=null"

# Get the raw content from the page
with urllib.request.urlopen(home_url) as url:
    s = url.read()

soup = BeautifulSoup(s, 'html.parser')

# Parse the content
url_section = 0
url_years = {}
url_bachelor = {}
url_master = {}

#For loop on all the possible fields with <option> to get the value associated with each year, or each "Bachelor" semester
for link in soup.find_all('option'):
    if link.text == 'Informatique':
        url_section = link.get('value')
    if "20" in link.text:
        #print(link.text)
        url_years[link.text] = link.get('value')
    if "Bachelor" in link.text:
        #print(link.text)
        url_bachelor[link.text] = link.get('value')
        #get('value'))
    if 'Master' in link.text or 'Mineur' in link.text:
        url_bachelor[link.text] = link.get('value')

        
print(url_years)
print(url_bachelor)
print(url_master)
#frame= frame.drop([2,3,4,5,6,8,9,11],axis=1)
#frame.head(10)

{'2010-2011': '39486325', '2016-2017': '355925344', '2014-2015': '213637922', '2012-2013': '123456101', '2011-2012': '123455150', '2007-2008': '978181', '2009-2010': '978195', '2015-2016': '213638028', '2013-2014': '213637754', '2008-2009': '978187'}
{'Projet Master automne': '249127', 'Master semestre 1': '2230106', 'Bachelor semestre 2': '249114', 'Master semestre 3': '2230128', 'Mineur semestre 1': '2335667', 'Mineur semestre 2': '2335676', 'Bachelor semestre 5b': '2226768', 'Master semestre 4': '2230140', 'Master semestre 2': '942192', 'Bachelor semestre 6': '942175', 'Bachelor semestre 4': '942163', 'Bachelor semestre 5': '942120', 'Bachelor semestre 1': '249108', 'Bachelor semestre 3': '942155', 'Bachelor semestre 6b': '2226785', 'Projet Master printemps': '3781783'}
{}


Function that replaces the general fields in the base_url with the fields for a given year and level of study

In [5]:
def getFullUrl(PeriodeAcad, PeriodePedag):
    url = base_url
    url = url.replace('[UNITE_ACADEMIQUE]', str(url_section))
    url = url.replace('[PERIODE_ACADEMIQUE]', str(url_years[PeriodeAcad]))
    url = url.replace('[PERIODE_PEDAGOGIQUE]', str(url_bachelor[PeriodePedag]))
    return url

print(getFullUrl('2007-2008', 'Bachelor semestre 1'))


http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=978181&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=null


 - We are interested only in the students that started between 2007-2010, in order to be sure that they will have completed their bachelor studies before or in 2016. It avoids our results getting biased by students actually still in studies (e.g. if we count students from 2013, we would only count students who finished their bachelor in 3 years, not actually those still in studies).
 - In order to have no bias at all, we will also have to check the records from the year 2006-2007,in order to verify whether the students actually are starting their studies in 2007-2008 or have already failed first year once before.

In [55]:
## Load all student from that have started EPFL from 2007 in current_student DataFrame
# We set the Sciper as index, as it is uniquely identifying each student.

current_student = pd.DataFrame(columns = ['Civilité', 'Nom Prénom', 'Starting year'])
current_student['Starting year']= current_student['Starting year'].astype(int)
current_student.index.name = 'No Sciper'

# We iterate over 3 years : 2008,2009,2010 -> We make sure the students all will have finished by 2016,
# as doing a bachelor can take only up to 6 years.
for year in range(2008,2011):
    year_string = str(year) + '-' + str(year+1)
    
    # Get the data from the appropriate url -> the URL describing the students in first semester
    current_url_B1 = getFullUrl(year_string, 'Bachelor semestre 1')
    with urllib.request.urlopen(current_url_B1) as url:
        html_B1 = url.read()
    soup = BeautifulSoup(html_B1, 'html.parser')
    
    # Create the data frame only keeping relevant fields
    # Note that the student_frame_B1 DataFrame is a temporary dataframe that only orders the data from the webpage.
    # We clean it and put all the non duplicate students into the current student DataFrame, that will contain our final entries
    student_frame_B1 = pd.read_html(soup.prettify(), header=1)
    student_frame_B1 = student_frame_B1[0].drop(0,axis=0)
    student_frame_B1 = student_frame_B1[['Civilité', 'Nom Prénom', 'No Sciper']]
    student_frame_B1 = student_frame_B1.set_index('No Sciper')
    student_frame_B1['Starting year'] = int(year);
    
    # Only keep students that are in their first 1st year
    student_frame_B1 = student_frame_B1[[not (sciper in current_student.index) for sciper in student_frame_B1.index]]
    
    current_student = pd.concat([current_student, student_frame_B1])
    
# We now check for all students starting in 2008 whether they have previously failed in year 2007-2008.
# If so, we change their starting year.
year_string = '2007-2008'
# Get the data from the appropriate url -> the URL describing the students in first semester
url_B1_2007 = getFullUrl(year_string, 'Bachelor semestre 1')
with urllib.request.urlopen(url_B1_2007) as url:
    html_B1_2007 = url.read()
soup = BeautifulSoup(html_B1_2007, 'html.parser')
student_frame_B1_2007 = pd.read_html(soup.prettify(), header=1)
sciper_student_2007 = np.array(student_frame_B1_2007[0]['No Sciper'])
current_student.loc[current_student.index.isin(sciper_student_2007), 'Starting year'] = 2007

    
print(current_student.index.is_unique)
print(current_student.head(100))

True
           Civilité                    Nom Prénom  Starting year
No Sciper                                                       
179383     Monsieur                    Amiet Nils           2008
186836     Monsieur         Andenmatten Alexandre           2008
184772     Monsieur                 Arnfred Jonas           2008
187235     Monsieur       Badila Constantin-Tudor           2008
186264     Monsieur                Bougacha Malik           2008
185949     Monsieur              Bourgeois Jérémy           2008
184264     Monsieur                  Bourqui Marc           2008
186825     Monsieur                Bruchez Lionel           2008
186009     Monsieur                   Bullot Marc           2008
179864     Monsieur              Cambazard Thomas           2007
184991     Monsieur              Chablais Michael           2008
185283     Monsieur             Constantin Victor           2008
178287     Monsieur               Cressier Gilles           2008
185952     Monsieur 

In [56]:
## and all students that have finished from 2007 in finished_student data frame


finished_student = pd.DataFrame(columns = ['Civilité', 'Nom Prénom', 'Finishing year'])
finished_student.index.name = 'No Sciper'

for year in range(2010,2017):
    year_string = str(year) + '-' + str(year+1)
    
    # Get the data from the appropriate url -> The URL containing the students in last semester.
    current_url_B6 = getFullUrl(year_string, 'Bachelor semestre 6')
    with urllib.request.urlopen(current_url_B6) as url:
        html_B6 = url.read()
    soup = BeautifulSoup(html_B6, 'html.parser')
    
    # Create the data frame only keeping relevant fields
    student_frame_B6 = pd.read_html(soup.prettify(), header=1)
    student_frame_B6 = student_frame_B6[0].drop(0,axis=0)
    student_frame_B6 = student_frame_B6[['Civilité', 'Nom Prénom', 'No Sciper']]
    student_frame_B6 = student_frame_B6.set_index('No Sciper') 
    # If we find a duplicate into the finished_student array, i.e. a student that was already in bachelor 6
    # the year before, it means that he failed his year and that we need to update his finishing year
    finished_student = finished_student[[not (sciper in student_frame_B6.index) for sciper in  finished_student.index]]
    student_frame_B6['Finishing year'] = int(year+1); # The +1 here gives us the finishing year (i.e. 2008 for the period 2007-2008)
    finished_student = pd.concat([finished_student, student_frame_B6])
        

print(finished_student.index.is_unique)
print(finished_student.head(5))


True
           Civilité          Nom Prénom  Finishing year
No Sciper                                              
174905     Monsieur     Aubelle Flavien            2011
179406     Monsieur  Baeriswyl Jonathan            2011
179428     Monsieur     Barroco Michael            2011
179426     Monsieur          Bloch Remi            2011
185949     Monsieur    Bourgeois Jérémy            2011


## Assignment 1

Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both `Bachelor semestre 1` and `Bachelor semestre 6`. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?

Note that we assume here that **a student reaching bachelor 6 and disappearing after in the dataset has successfully completed his bachelor**, as there is absolutely no way to be sure he actually did so. We cannot check the first master year because he might change section/university, ... 

In [58]:
finished_student_sciper = finished_student.index

# Only keep students that started after 2007.
finished_student_from_2007 = current_student.loc[finished_student_sciper]

# Merge the two dataFrames so we get all the students that started and finished. 
# The number of dropouts could be computed by taking the difference between everyone that 
# entered in first year and does not appear at the end (Note that we wouldn't count the people who changed their section).
sample_student = pd.merge(current_student, finished_student)

#Compute the duration it took to complete the bachelor and group by sex, then compute the average.
sample_student['Bachelor duration'] = sample_student['Finishing year']-sample_student['Starting year']
sample_student.groupby('Civilité')['Bachelor duration'].mean()

#There is yet to conclude whether it is statistically significant or not.

print(sample_student)

     Civilité                         Nom Prénom  Starting year  \
0    Monsieur                     Bougacha Malik           2008   
1    Monsieur                   Bourgeois Jérémy           2008   
2    Monsieur                       Bourqui Marc           2008   
3    Monsieur                     Bruchez Lionel           2008   
4    Monsieur                   Chablais Michael           2008   
5    Monsieur                  Constantin Victor           2008   
6    Monsieur                    Cressier Gilles           2008   
7    Monsieur                   Cuony Maximilien           2008   
8    Monsieur                     Droxler Thomas           2008   
9    Monsieur                        Fiaux Anaël           2008   
10   Monsieur                        Fol Charles           2008   
11   Monsieur                      Fond Matthieu           2008   
12   Monsieur                  Franscella Mattia           2008   
13   Monsieur                       Gardiol Loïc           200

In [83]:
# Statistical test
import scipy.stats as stats
import math

current_student.loc[current_student.index.isin(sciper_student_2007), 'Starting year']

men_student = sample_student.loc[sample_student['Civilité']=='Monsieur']
women_student = sample_student.loc[sample_student['Civilité']=='Madame']
print(women_student)
#print(women_student.columns)
print(women_student.loc[:,'Bachelor duration'])

print(men_student.loc[:,'Bachelor duration'].mean())
print(women_student.loc[:,'Bachelor duration'].mean())
print(men_student.loc[:,'Bachelor duration'].std())
print(women_student.loc[:,'Bachelor duration'].std())

stats.ttest_ind(a= np.array(women_student.loc[:,'Bachelor duration']),
                b= np.array(men_student.loc[:,'Bachelor duration']),
                equal_var=False)    # Assume samples have equal variance?

    Civilité                  Nom Prénom  Starting year  Finishing year  \
16    Madame              Heldner Céline           2008            2011   
26    Madame            Rodriguez Oriane           2008            2011   
29    Madame             Salvisberg Véra           2008            2011   
33    Madame                 Sisto Maria           2008            2012   
36    Madame      Triponez Elodie Nilane           2008            2011   
46    Madame           Balmau Oana Maria           2009            2012   
75    Madame             Rollier Orianne           2009            2013   
99    Madame         Colussi Maëlle Lise           2010            2013   
128   Madame  Musso Claire Cécile Hélène           2010            2013   

     Bachelor duration  
16                   3  
26                   3  
29                   3  
33                   4  
36                   3  
46                   3  
75                   4  
99                   3  
128                  3  

Ttest_indResult(statistic=-2.7350760802265679, pvalue=0.017034663994017443)

Assignment 2 : Master students